In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import re
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sn
import pickle
import matplotlib.pyplot as plt
import jieba
from gensim.models import Word2Vec
import multiprocess

import ast

from nltk.corpus import stopwords
import nltk

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /shared_home/willie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
stopWords = stopwords.words('english')

In [3]:
df = pd.read_excel('LoughranMcDonald_MasterDictionary_2018.xlsx')
df.head()

,Word,Sequence Number,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Constraining,Superfluous,Interesting,Modal,Irr_Verb,Harvard_IV,Syllables,Source
0,AARDVARK,1,277,1.480368e-08,1.239377e-08,3.564730e-06,84,0,0,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.603287e-10,9.725110e-12,9.863549e-09,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,8,4.275431e-10,1.386497e-10,6.225591e-08,7,0,0,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,12,6.413147e-10,3.159061e-10,9.383557e-08,12,0,0,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,7250,3.874610e-07,3.681624e-07,3.366553e-05,914,0,0,0,0,0,0,0,0,0,0,3,12of12inf


In [4]:
score = pd.read_csv('./sasa/score_QK_v2.csv')
score.head()

,Unnamed: 0,company,date,return,bad,url,QK,score
0,0,A,2000-03-15,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q,"[508, 176, 250, 304, 63, 143, 116]"
1,1,A,2000-06-12,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q,"[494, 133, 223, 278, 42, 133, 97]"
2,3,A,2001-01-17,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K,"[569, 172, 231, 284, 47, 126, 127]"
3,4,A,2001-03-19,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q,"[449, 121, 200, 232, 40, 123, 111]"
4,5,A,2001-06-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q,"[519, 123, 209, 225, 47, 130, 107]"


In [ ]:
filenames = [f'./item2_{i}_QK.csv' for i in range(4)]
datas = []
for f in filenames:
    datas.append(pd.read_csv(f))
temp = pd.concat(datas)

In [ ]:
temp.to_csv('item2_text_QK.csv',index=False)

In [6]:
key_words = df['Word'].values

In [5]:
def add_key_words_to_jieba(key_words):
    for word in key_words:
        try:
            word = word.lower()
            jieba.add_word(word)
        except:
            print(word)

In [8]:
add_key_words_to_jieba(key_words)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.706 seconds.
Prefix dict has been built succesfully.


nan


In [9]:
punct = set(u''':!),$=.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒
﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠
々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻
︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')

In [6]:
def process_sentence(sentence,stopWords,punct):
    cut_sentence = ' '.join(jieba.cut(sentence))
    split_words = cut_sentence.split(' ')
    
    # Remove None
    rmNone_words = list(filter(None, split_words))
    
    # Remove stopwords
    rmStopWords_words = list(filter(lambda a: a not in stopWords and a != '\n', rmNone_words))
    
    # Remove punctuation
    filterpuntl = lambda l: list(filter(lambda x: x not in punct, l))
    rmPunc_words = filterpuntl(rmStopWords_words)
    
    return rmPunc_words

In [15]:
rmPunc_words = process_sentence(temp['text'].values[0],stopWords,punct)

In [16]:
rmPunc_words

['item',
 '2',
 'management',
 'discussion',
 'analysis',
 'financial',
 'condition',
 'results',
 'operations',
 'following',
 'discussion',
 'read',
 'conjunction',
 'condensed',
 'consolidated',
 'financial',
 'statements',
 'notes',
 'thereto',
 'included',
 'elsewhere',
 'form',
 '10',
 'q',
 'following',
 'discussion',
 'contains',
 'forward',
 'looking',
 'statements',
 'involve',
 'risks',
 'uncertainties',
 'use',
 'words',
 'anticipates',
 'believes',
 'plans',
 'expects',
 'future',
 'intends',
 'may',
 'estimates',
 'predicts',
 'potential',
 'continue',
 'similar',
 'expressions',
 'identify',
 'forward',
 'looking',
 'statements',
 'actual',
 'results',
 'could',
 'differ',
 'materially',
 'results',
 'contemplated',
 'forward',
 'looking',
 'statements',
 'due',
 'certain',
 'factors',
 'including',
 'discussed',
 'factors',
 'may',
 'affect',
 'future',
 'results',
 'elsewhere',
 'form',
 '10',
 'q',
 'overview',
 'march',
 '2',
 '1999',
 'hew',
 'let',
 'pack',
 'ard',

In [8]:
def get_all_cut_sentence(input_):
    item2_df_text,stopWords,punct,ver = input_
#     cut_sentences = []
    with open(f'cut_sentences_{ver}_15000.pkl','rb') as f:
        cut_sentences  = pickle.load(f)
    for i,text in enumerate(item2_df_text):
        if(i % 5000 == 0):
            print(i)
        if(i <= 15000):
            continue
        else:
            cut_sentences.append(process_sentence(text,stopWords,punct))
            if(i % 5000 == 0):
                with open(f'cut_sentences_{ver}_{i}.pkl','wb') as f:
                    pickle.dump(cut_sentences,f)
    with open(f'cut_sentences_{ver}_{i}.pkl','wb') as f:
        pickle.dump(cut_sentences,f)
    return cut_sentences

def filter_scoresum(df,threshold):
    scoresum_list = []
    df_score = df['score'].values
    for score in df_score:
        scoresum_list.append(sum(ast.literal_eval(score)))
    df['scoresum'] = scoresum_list
    return df[df['scoresum'] > threshold].copy()

In [18]:
filter_score = filter_scoresum(temp,100)

In [21]:
filter_score.head()

,Unnamed: 0,company,date,return,bad,url,QK,text,score,scoresum
0,0,A,2000-03-15,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q,item 2. management's discussion and analysis o...,"[508, 176, 250, 304, 63, 143, 116]",1560
1,1,A,2000-06-12,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q,item 2. management's discussion and analysis o...,"[494, 133, 223, 278, 42, 133, 97]",1400
2,3,A,2001-01-17,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K,item 7. management's discussion and analysis o...,"[569, 172, 231, 284, 47, 126, 127]",1556
3,4,A,2001-03-19,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q,item 2. management's discussion and analysis o...,"[449, 121, 200, 232, 40, 123, 111]",1276
4,5,A,2001-06-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q,item 2. management's discussion and analysis o...,"[519, 123, 209, 225, 47, 130, 107]",1360


In [22]:
filter_score.shape[0]/4

35265.75

In [ ]:
length = filter_score.shape[0]
all_text = filter_score['text'].values
quarter = int(length/4)

text_0, text_1, text_2, text_3 = all_text[:quarter], all_text[quarter:2*quarter],\
                                                all_text[quarter*2:quarter*3], all_text[quarter*3:]
works = [(text_0,stopWords,punct,1),(text_1,stopWords,punct,2),(text_2,stopWords,punct,3),(text_3,stopWords,punct,4)]
ans = []
if __name__ == '__main__':
    pool = multiprocess.Pool(processes=4)
    ans.append(pool.map(get_all_cut_sentence,works))

0
0
5000
10000
15000
5000
10000
15000
0
5000
10000
15000
0
5000
10000
15000
20000
20000
20000
20000
25000
25000
25000
25000
30000
30000
30000
30000
35000
35000
35000
35000


In [ ]:
all_cut_sentences = ans[0][0]+ans[0][1]+ans[0][2]+ans[0][3]

In [9]:
import pickle

In [ ]:
# with open('all_cut_sentences.pkl','wb') as f:
#     pickle.dump(all_cut_sentences,f)
with open('all_cut_sentences.pkl','rb') as f:
    all_cut_sentences = pickle.load(f)

In [10]:
filenames = ['cut_sentences_2_35000.pkl','cut_sentences_1_35000.pkl','cut_sentences_3_35264.pkl','cut_sentences_4_35267.pkl']

In [12]:
datas = []
for f in filenames:
    with open(f,'rb') as f:
        datas.append(pickle.load(f))

In [16]:
all_cut_sentences = datas[0]+datas[1]+datas[2]+datas[3]

### Word2vec

In [17]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(all_cut_sentences,iter=20,size=300,sg=1) # skip gram, vector size=250
model.save('word2vec.model')

In [26]:
model_CBOW = Word2Vec(all_cut_sentences,iter=10,size=300) # skip gram, vector size=250
model_CBOW.save('word2vec_CBOW.model')

In [78]:
key_words[30].lower()

'abatement'

In [27]:
word = key_words[30].lower()
topn = 10
pd.DataFrame(model_CBOW.wv.most_similar(word, topn=topn), columns=[word, 'cos'])

,abatement,cos
0,abatements,0.516063
1,occupation,0.403656
2,holdover,0.395056
3,underpayment,0.374960
4,removal,0.374015
5,vacancies,0.367847
6,vacating,0.363724
7,purta,0.363636
8,refunds,0.355907
9,disallowance,0.354710


In [19]:
word = key_words[30].lower()
topn = 10
pd.DataFrame(model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])

NameError: name 'key_words' is not defined

In [20]:
embedding_matrix = np.zeros((len(model.wv.vocab.items()) + 1, model.vector_size))
word2idx = {}
vocab_list = [(word, model.wv[word]) for word, _ in model.wv.vocab.items()]
for i, vocab in enumerate(vocab_list):
    word, vec = vocab
    embedding_matrix[i + 1] = vec
    word2idx[word] = i + 1

### Count words for each sentiment

In [21]:
df_sen = pd.read_excel('LoughranMcDonald_SentimentWordLists_2018.xlsx',sheet_name = None)
sentiment_names = ['Negative','Positive','Uncertainty','Litigious','StrongModal','WeakModal','Constraining']

In [22]:
[i.lower() for i in df_sen['Negative'].dropna().values.reshape((-1))]

['abandoned',
 'abandoning',
 'abandonment',
 'abandonments',
 'abandons',
 'abdicated',
 'abdicates',
 'abdicating',
 'abdication',
 'abdications',
 'aberrant',
 'aberration',
 'aberrational',
 'aberrations',
 'abetting',
 'abnormal',
 'abnormalities',
 'abnormality',
 'abnormally',
 'abolish',
 'abolished',
 'abolishes',
 'abolishing',
 'abrogate',
 'abrogated',
 'abrogates',
 'abrogating',
 'abrogation',
 'abrogations',
 'abrupt',
 'abruptly',
 'abruptness',
 'absence',
 'absences',
 'absenteeism',
 'abuse',
 'abused',
 'abuses',
 'abusing',
 'abusive',
 'abusively',
 'abusiveness',
 'accident',
 'accidental',
 'accidentally',
 'accidents',
 'accusation',
 'accusations',
 'accuse',
 'accused',
 'accuses',
 'accusing',
 'acquiesce',
 'acquiesced',
 'acquiesces',
 'acquiescing',
 'acquit',
 'acquits',
 'acquittal',
 'acquittals',
 'acquitted',
 'acquitting',
 'adulterate',
 'adulterated',
 'adulterating',
 'adulteration',
 'adulterations',
 'adversarial',
 'adversaries',
 'adversary',

In [23]:
def get_extradic(df_sen,sentiment_names,model):
    dic_sentiment_extra = []
    
    # each sentiment
    for sentiment in sentiment_names:
        sentiment_words = [i.lower() for i in df_sen[sentiment].dropna().values.reshape((-1))]
        extra_words = {}
        
        # each sentiment words
        for word in sentiment_words:
            # there is the word in word2vec model
            try:
                similar_words = list(filter(lambda x: x not in sentiment_words,[tu[0] for tu in model.wv.most_similar(word, topn=10)]))
                
                for s_word in similar_words:
                    # each silimiar words
                    if s_word in extra_words:
                        extra_words[s_word] += 1
                    else:
                        extra_words[s_word] = 1
            # not contain the word
            except:
                print('Not contain',word)
        dic_sentiment_extra.append(extra_words)
    return dic_sentiment_extra

In [24]:
dic_sentiment_extra = get_extradic(df_sen,sentiment_names,model)

Not contain abdicated
Not contain abdicates
Not contain abdicating
Not contain abdication
Not contain abdications
Not contain aberrational
Not contain abrogates
Not contain abrogating
Not contain abrogations
Not contain abruptness
Not contain abusively
Not contain abusiveness
Not contain acquiesce
Not contain acquits
Not contain acquittal
Not contain acquittals
Not contain acquitting
Not contain adulterate
Not contain adulterating
Not contain aftermaths
Not contain against
Not contain aggravations
Not contain alienated
Not contain alienates
Not contain alienations
Not contain annoy
Not contain annoyances
Not contain annoyed
Not contain annoying
Not contain annoys
Not contain annulling
Not contain annulments
Not contain annuls
Not contain anomalously
Not contain argumentative
Not contain assaulted
Not contain assaulting
Not contain balked
Not contain bankrupting
Not contain bankrupts
Not contain boycotting
Not contain breakages
Not contain briberies
Not contain calamitous
Not contain ca

Not contain unsuitably
Not contain unsuited
Not contain unsuspected
Not contain untruth
Not contain untruthful
Not contain untruthfully
Not contain untruthfulness
Not contain untruths
Not contain unwelcome
Not contain usurious
Not contain usurp
Not contain usurped
Not contain usurping
Not contain usurps
Not contain vitiate
Not contain vitiated
Not contain vitiates
Not contain vitiating
Not contain vulnerably
Not contain worrying
Not contain beautifully
Not contain creativeness
Not contain delightfully
Not contain enjoyably
Not contain excelling
Not contain exclusiveness
Not contain exclusives
Not contain greatness
Not contain happily
Not contain happiness
Not contain impresses
Not contain impressing
Not contain innovativeness
Not contain pleasantly
Not contain proficiently
Not contain prospering
Not contain prospers
Not contain spectacularly
Not contain abeyances
Not contain anomalously
Not contain arbitrariness
Not contain doubted
Not contain improbability
Not contain inexactness
Not 

In [25]:
for i,dic in enumerate(dic_sentiment_extra):
    print(sentiment_names[i],len(dic))

Negative 6528
Positive 1744
Uncertainty 1731
Litigious 3170
StrongModal 156
WeakModal 167
Constraining 978


In [26]:
def extract_importance_sentimentwords(dic_sentiment_extra,sentiment_names,threshold = 10):
    dic_sentiment = {}
    for i,dic in enumerate(dic_sentiment_extra):
        dic_sentiment[sentiment_names[i]] = []
        temp = sorted(dic.items(),key=lambda kv: kv[1],reverse=True)
        filter_words = list(filter(lambda x:x[1]>=threshold,temp))
        for word in filter_words:
            dic_sentiment[sentiment_names[i]].append(word[0].upper())
        dic_sentiment[sentiment_names[i]] = np.array(dic_sentiment[sentiment_names[i]])
    return dic_sentiment

In [27]:
dic_final_sentiment = extract_importance_sentimentwords(dic_sentiment_extra,sentiment_names)

In [28]:
dic_final_sentiment

{'Negative': array(['APPEALED', 'TAMPERING', 'INTERLOCUTORY', 'WANTON', 'OSCS',
        'PUNISHABLE', 'LAWSUIT', 'RULED', 'VIGOROUSLY', 'VICTIMIZED',
        'MOOT', 'RANSOMWARE', 'WILLINGLY', 'BASELESS', 'DECLARATORY',
        'CIVILLY', 'WRITS', 'AGGRIEVED', 'HESITANT', 'LAWSUITS',
        'VOIDABLE', 'SANCTIONS', 'CODEFENDANT', 'EXITING', 'PUNISH',
        'PERPETRATORS', 'ADJUDGED', 'OVERSTEPPED', 'CLRA', 'PROCEEDING',
        'HIDING', 'ASSERTED', 'OVERRULED', 'MOREOVER', 'ALTOGETHER',
        'SOFTNESS', 'REHEAR', 'TORTIOUS', 'INFLICTION', 'REPUTATION',
        'PREEMPTS', 'FRISK', 'COUNTERSUED', 'RESOLVED', 'COURT',
        'UNPRECEDENTED', 'PATENTS', 'TRICKERY', 'ARBITRABLE', 'ASSERTING',
        'OPPRESSIVE', 'TERROR', 'SEEN', 'LEGITIMATE', 'REASSERTED',
        'REPLEAD', 'CORRECT', 'ECONOMY', 'PREVENT', 'CHRONIC', 'PATENT',
        'INFRINGER', 'SOARING', 'MOTIONS', 'APPELLATE', 'ABETTED',
        'HACKING', 'CYBER', 'MIRED', 'WRONGS', 'TREBLE', 'INJUNCTIVE',
        'MUTED'

In [29]:
with open('dic_final_extra_sentiment.pkl','wb') as f:
    pickle.dump(dic_final_sentiment,f)

### Prepare Bert data

In [24]:
score['url'][1500]

'/Archives/edgar/data/2135/000095013406002343/d32774e10vq.htm'

In [6]:
text_list = score.loc[:,'text'].values

In [28]:
def get_trainData(score,filename1,filename2):
    label_list = score.loc[:,'bad'].values.reshape((-1,1))
    text_list = score.loc[:,'text'].values.reshape((-1,1))
    train_data = np.concatenate([text_list,label_list],axis = 1)
    
    np.random.shuffle(train_data)
    length = int(train_data.shape[0]*0.8)
    with open(filename1,'wb') as f:
        pickle.dump(train_data[:length,:],f)
    with open(filename2,'wb') as f:
        pickle.dump(train_data[length:,:],f)

In [29]:
get_trainData(score,'train.pkl','eval.pkl')